In [1]:
'''
回測策略說明:
布林通道(20MA)三倍標準差
買點:

賣點:

停損:

停利:

'''

'\n回測策略說明:\n布林通道(20MA)三倍標準差\n買點:\n\n賣點:\n\n停損:\n\n停利:\n\n'

In [1]:
'''
環境設定
'''
import sys
!conda install --yes --prefix {sys.prefix} requests
!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} pandas

done

# All requested packages already installed.

done

# All requested packages already installed.

done

# All requested packages already installed.



In [4]:
'''
爬蟲抓取目標範圍資料
'''
import os
import csv
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
import requests
import re
import time
from functools import reduce

time_interval = 3
target = '2330'
start_from = '2017-01-01'
root = os.path.abspath(os.path.join('..'))
target_filepath = f'{root}/data/{target}.csv'

csv_file = Path(target_filepath)
columns = [
    '日期',
    '成交股數',
    '成交金額',
    '開盤價',
    '最高價',
    '最低價',
    '收盤價',
    '漲跌價差',
    '成交筆數' 
]
if not csv_file.is_file():
    with open(target_filepath, 'w',  newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(columns)

'''
列出範圍裡面的日期, 從最新的月份往回跑
'''
date_from = datetime.strptime(start_from, '%Y-%m-%d')
date_to = datetime.now()
date_list = []
while date_from <= date_to:
    date_str = date_from.strftime('%Y%m01')
    if date_str not in date_list:
        date_list.append(date_str)
    date_from = date_from + timedelta(hours=24)

date_list.reverse()


data_frame = pd.read_csv(f'{root}/data/{target}.csv', sep=",", header=0, usecols=columns)

'''
ref: https://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY.html
爬蟲抓取個股月成交資訊
'''
'''
@TODO: 嘗試避免打已經抓完整個月份的 http request
'''
for date_str in date_list:
    rows = []
    url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date={date_str}&stockNo={target}'
    print('url: %s' % url)
    response = requests.get(url, allow_redirects=True)
    if response.status_code is 200:
        content = response.content.decode('cp1252')
        rows = list(filter(lambda o: re.search("(\d+)\/(\d+)\/(\d+)", o), content.split('\r\n')))
        rows = list(map(lambda o: list(map(lambda s: s.replace('"','').replace(',','') , o.split('","'))), rows))

        def remapToDcit(x, y):
            row = {}
            for index, col in enumerate(columns):
                row[col] = y[index]
            x.append(row)
            return x
            
        rows = reduce(remapToDcit, rows, [])
        twse_soure = pd.DataFrame([], columns=columns)
        '''
        判斷爬回的資料有沒有在csv裡面
        '''
        for row in rows:
            is_existed = True if len(list(filter(lambda o: o[0] == row['日期'], data_frame.values))) else False
            if not is_existed:
                twse_soure.append([row])

        data_frame = data_frame.append(twse_soure)
        time.sleep(time_interval)

    else:
        raise IOError(f'Network Error, url: {url} status code: {response.status_code}')
# 資料排序

# 存入 CSV
data_frame = data_frame.drop_duplicates(keep=False, ignore_index=True).sort_values(by='日期')
data_frame.to_csv(target_filepath, sep=',', encoding='utf-8', index=False) 


'''
布林通道邏輯
'''
source = data_frame.loc[:,['收盤價']]
df_20ma = source.rolling(window=20).mean().rename(columns={ '收盤價': '20ma' })
df_std = source.rolling(window=20).std().rename(columns={ '收盤價': 'std' })

def transform2Dto1D(x, y):
    x.append(y[0])
    return x
'''
@TODO: 嘗試對做出panda 結果物件最佳化
'''
result = pd.DataFrame({
    '日期': reduce(transform2Dto1D, data_frame.loc[:,['日期']].values, []),
    '收盤價': reduce(transform2Dto1D, data_frame.loc[:,['收盤價']].values, []),
    '20ma': reduce(transform2Dto1D, df_20ma.values, []),
    'std': reduce(transform2Dto1D, df_std.values, [])
})
result['bollinger_down'] = result['20ma'] - 3 * result['std']
result['bollinger_up'] = result['20ma'] + 3 * result['std']
result['should_short'] = result['收盤價'] > result['bollinger_up']
print(result)
'''
報表
'''
'''
回測
'''

'''
畫圖
'''

url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20200401&stockNo=2330
url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20200301&stockNo=2330
url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20200201&stockNo=2330
url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20200101&stockNo=2330
url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20191201&stockNo=2330
url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20191101&stockNo=2330
url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20191001&stockNo=2330
url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20190901&stockNo=2330
url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20190801&stockNo=2330
url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date=20190701&stockNo=2330
url: https://www.twse.com.tw/exchangeReport/STOCK_DAY?respon

'\n畫圖\n'